In [2]:
import json
from datetime import datetime
from typing import List, Dict, Any

import openai
from dotenv import load_dotenv
load_dotenv()
import os

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

In [3]:
# 1. 모든 JSON 파일을 로드하고 content/date를 통일된 리스트로 합침
def load_all_chunks(json_paths: List[str]) -> List[Dict[str, Any]]:
    all_chunks = []
    for path in json_paths:
        with open(path, encoding="utf-8") as f:
            data = json.load(f)
            for item in data:
                content = item.get("content") or item.get("text")
                date = item.get("date")
                if content and date:
                    all_chunks.append({"content": content, "date": date})
    print(f"[1] 총 청크 개수: {len(all_chunks)}")
    print(f"[1] 예시 청크: {all_chunks[0] if all_chunks else '없음'}")
    assert all_chunks, "청크가 비어있음"
    return all_chunks

In [4]:
# 2. 날짜를 파싱해서 정렬에 쓸 수 있도록 변환
def parse_date(date_str: str) -> datetime:
    for fmt in ("%Y.%m.%d", "%y.%m.%d", "%Y-%m-%d"):
        try:
            return datetime.strptime(date_str, fmt)
        except Exception:
            continue
    return datetime(1900, 1, 1)

In [5]:
# 3. 임베딩 모델 준비 (OpenAI API 사용)
class OpenAIEmbedder:
    def __init__(self, api_key, model="text-embedding-3-large"):
        self.api_key = api_key
        self.model = model

    def encode(self, texts, show_progress_bar=False, convert_to_numpy=True):
        openai.api_key = self.api_key  # 항상 최신 키로 할당
        embeddings = []
        for text in texts:
            response = openai.embeddings.create(
                input=text,
                model=self.model
            )
            emb = response.data[0].embedding
            embeddings.append(emb)
        if convert_to_numpy:
            import numpy as np
            return np.array(embeddings)
        return embeddings

In [6]:
# 실제 API 키를 환경변수에서 불러오기
api_key = os.getenv("OPENAI_API_KEY")
embedder = OpenAIEmbedder(api_key=api_key, model="text-embedding-3-large")

import faiss
import numpy as np

In [7]:
# 4. 모든 청크 임베딩 및 FAISS 벡터스토어 구축
def build_faiss_index(chunks: List[Dict[str, Any]]):
    texts = [c["content"] for c in chunks]
    metadatas = [{"date": c["date"]} for c in chunks]
    embeddings = embedder.encode(texts, show_progress_bar=True, convert_to_numpy=True)
    print(f"[2] 임베딩 shape: {embeddings.shape}")
    assert embeddings.shape[0] == len(texts), "임베딩 개수 불일치"
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    print(f"[3] FAISS 인덱스 벡터 개수: {index.ntotal}")
    assert index.ntotal == len(texts), "FAISS 인덱스 개수 불일치"
    return index, embeddings, texts, metadatas

In [8]:
# 5. 쿼리 임베딩 후 FAISS로 유사 청크 top-k 검색
def search_faiss(query: str, index, embeddings, texts, metadatas, top_k=5):
    q_emb = embedder.encode([query], convert_to_numpy=True)  # 수정된 부분
    D, I = index.search(q_emb, top_k)
    results = []
    for idx in I[0]:
        if idx < len(texts):
            results.append({
                "content": texts[idx],
                "date": metadatas[idx]["date"],
                "parsed_date": parse_date(metadatas[idx]["date"])
            })
    results = sorted(results, key=lambda x: x["parsed_date"], reverse=True)
    print(f"[4] 쿼리 '{query}' top-{top_k} 검색 결과:")
    for r in results:
        print(f"  - [{r['date']}] {r['content'][:40]}...")
    assert results, "검색 결과 없음"
    return results

In [9]:
# 6. LLM 준비 (OpenAI API 키 필요)
llm = ChatOpenAI(
    temperature=0.2,
    model="gpt-3.5-turbo",
    openai_api_key=api_key
)


In [10]:
# 7. 챗봇 히스토리 관리 및 답변 생성
class CBAMChatbot:
    def __init__(self, index, embeddings, texts, metadatas):
        self.index = index
        self.embeddings = embeddings
        self.texts = texts
        self.metadatas = metadatas
        self.history = []
    
    def ask(self, user_query: str):
        relevant_chunks = search_faiss(user_query, self.index, self.embeddings, self.texts, self.metadatas, top_k=5)
        context = "\n\n".join([f"[{c['date']}] {c['content']}" for c in relevant_chunks])
        history_prompt = ""
        for u, b in self.history[-5:]:
            history_prompt += f"User: {u}\nBot: {b}\n"
        prompt = (
            f"{history_prompt}"
            f"User: {user_query}\n"
            f"CBAM 관련 최신 정보와 문서를 참고하여 답변해 주세요.\n"
            f"참고 문서:\n{context}\n"
            f"Bot:"
        )
        # 프롬프트를 HumanMessage 형식으로 변환
        messages = [HumanMessage(content=prompt)]
        # invoke 메서드 사용
        answer = llm.invoke(messages)
        print(f"[5] LLM 답변: {answer[:100]}...")
        self.history.append((user_query, answer))
        return answer


In [11]:
# 8. 전체 파이프라인 실행 예시
if __name__ == "__main__":
    json_paths = [
        r"C:\bit_esg\python\LLM--main\cbam_chunk.json",
        r"C:\bit_esg\python\LLM--main\gl_chunk.json",
        r"C:\bit_esg\python\LLM--main\manual_chunk.json",
        r"C:\bit_esg\python\LLM--main\omnibus_chunk.json"
    ]
    all_chunks = load_all_chunks(json_paths)
    index, embeddings, texts, metadatas = build_faiss_index(all_chunks)
    chatbot = CBAMChatbot(index, embeddings, texts, metadatas)

    # 사용자 입력을 받아 질문 처리
    user_input = input("질문을 입력하세요: ")
    answer = chatbot.ask(user_input)
    print("답변:", answer)

[1] 총 청크 개수: 111
[1] 예시 청크: {'content': '구조\n개념 요약 : 해당 항목이 CBAM 제도 상 어떤 의미를 가지는지 간단히 정의\n플랫폼 반영 방식 : 실제 우리 플랫폼에서 그 개념이 어떻게 구현되었는지 설명\n보충 설명 : 실무자가 이해할 수 있도록 배경이나 예시 중심으로 풀어 설명', 'date': '23.08.17'}
[2] 임베딩 shape: (111, 3072)
[3] FAISS 인덱스 벡터 개수: 111
[4] 쿼리 '전구물질이 뭐야?' top-5 검색 결과:
  - [2025.02.26] 제조건이 되거나 이를 배제해서는 안 된다.자료: COM(2025)81(E...
  - [2025.02.26] 우리 정부의 탄소중립 정책도 산업 발전을 목표로 새롭게 설계될 필요가 있...
  - [23.08.17] 전구물질 배출량 산정 방법
개념 요약 : 옴니버스 패키지 발표 내용에 따...
  - [23.08.17] 단순재/복합재 여부 확인
개념 요약 : 전구물질 포함 여부에 따라 단순/...
  - [23.08.17] 생산공정 내 투입물질 확인
개념 요약 : 직접 배출량 산정을 위해 투입 ...


TypeError: 'AIMessage' object is not subscriptable